In [32]:
import os
from dotenv import load_dotenv
_ = load_dotenv()

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import AzureChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun

In [33]:
# Azure AI Foundry configuration
model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-12-01-preview",
    azure_deployment="gpt-4.1",  # Your deployment name
    model="gpt-4.1",
    temperature=0
)

In [34]:
model.invoke([SystemMessage(content="Ready for working?")]).content

"Yes, I'm ready to help! What do you need assistance with today?"

## Import Prompts

In [35]:
import sys
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Load the intent classifier prompt from the text file
with open(os.path.join(parent_dir, "prompts", "intent_classifier.txt"), "r") as f:
    intent_classifier_prompt = f.read()

intent_classifier_prompt

'You are an intent classifier for the ChatSupervisor agent, the central brain of a worker assistant. Your primary responsibility is to analyze the user\'s latest message in the context of the chat history and determine the user\'s primary intent.\n\nYour response must be only the name of the single, most appropriate agent or tool to handle this intent. Do not provide any other text, explanation, or preamble. Respond with the agent name on a single line, with no extra whitespace, punctuation, or formatting.\n\nThese are your only valid routing options:\n\n1.  InformationRetrievalAgent\n    - Purpose: To answer generic questions about work policies, procedures, or "how-to" guides using the internal knowledge base.\n    - Examples:\n        - "How do I call in sick?"\n        - "What are the company rules for overtime?"\n        - "How are reservations calculated?"\n\n2.  ActionExecutionAgent\n    - Purpose: To handle personalized requests that require fetching the user\'s own specific da

## Agent Set-up

In [36]:
# Agent State
class AgentState(TypedDict):
   messages: Annotated[list[AnyMessage], "Conversation messages"]
   next_action: Annotated[str, "Next action to take"]
   retrieved_data: Annotated[str, "Data retrieved from tools"]
   error_message: Annotated[str, "Error message if any"]
   attempts: Annotated[int, "Number of attempts made"]
   max_retries: Annotated[int, "Maximum number of retries allowed"]

In [37]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import StateGraph, END


class ChatSupervisorAgent:
    def __init__(self, model, checkpointer=None):
        self.model = model
        # The system prompt is the intent classifier
        self.system = intent_classifier_prompt

        graph = StateGraph(AgentState)

        # 1. Define the node that classifies intent
        graph.add_node("classify_intent", self.classify_intent_node)

        # 2. Define the conditional edge (the router)
        graph.add_conditional_edges(
            "classify_intent",  # Start from the classifier node
            self.route_intent,   # Use this function to find the next step
            {
                # Keys must match the strings from the prompt
                "ActionExecutionAgent": "action_execution_agent",
                "InformationRetrievalAgent": "information_retrieval_agent",
                "FallbackTool": "fallback_tool",
            }
        )
        
        # 3. Set the entry point for the graph
        graph.set_entry_point("classify_intent")

        # --- Define Specialist Nodes ---
        # graph.add_node("action_execution_agent", action_agent_runnable)
        # graph.add_node("information_retrieval_agent", retrieval_agent_runnable)
        # graph.add_node("fallback_tool", fallback_runnable)
        # graph.add_node("answer_agent", answer_agent_runnable)

        # --- Define Edges to AnswerAgent (Enforcing ToV) ---
        # As per the brief, all specialists must go to the AnswerAgent
        # graph.add_edge("action_execution_agent", "answer_agent")
        # graph.add_edge("information_retrieval_agent", "answer_agent")
        # graph.add_edge("fallback_tool", "answer_agent")
        # graph.add_edge("answer_agent", END) # End the graph run

        # 4. Compile the graph
        # self.graph = graph.compile(checkpointer=checkpointer)
        # print("Graph structure defined.")
    
    def classify_intent_node(self, state: AgentState):
        """
        Calls the LLM with the intent classifier prompt to decide the next step.
        Updates the 'next_node' field in the state.
        """
        print("---SUPERVISOR: CLASSIFYING INTENT---")
        
        # Prepare the prompt and messages
        messages = [HumanMessage(content=self.system)]
        messages.extend(state['messages'])

        # Call the model
        response = self.model.invoke(messages)
        intent = response.content.strip()

        # Failsafe: If LLM returns an invalid route, force fallback
        if intent not in ["ActionExecutionAgent", "InformationRetrievalAgent", "FallbackTool"]:
            print(f"Warning: LLM returned invalid intent '{intent}'. Forcing Fallback.")
            intent = "FallbackTool"

        print(f"Intent classified as: {intent}")
        
        # Update the state so the router can read it
        return {"next_node": intent}
    
    def route_intent(self, state: AgentState) -> str:
        """
        Reads the 'next_node' field from the state and returns it.
        This tells LangGraph which node to go to next.
        """
        intent = state.get("next_node")
        print(f"---SUPERVISOR: ROUTING TO: {intent}---")
        
        if not intent:
            return "FallbackTool" # Should not happen, but good failsafe
            
        return intent

In [38]:
agent = ChatSupervisorAgent(model=model)

In [39]:
# Test the agent with a sample message
test_state = {
    "messages": [HumanMessage(content="What is my payroll?")],
    "next_action": "",
    "retrieved_data": "",
    "error_message": "",
    "attempts": 0,
    "max_retries": 3
}

result = agent.classify_intent_node(test_state)

---SUPERVISOR: CLASSIFYING INTENT---
Intent classified as: ActionExecutionAgent
Intent classified as: ActionExecutionAgent
